# 1. Data Extraction

In this notebook we will extract our data and analyse it. For that purpose, we are importing from our library where we define the
```bcrp_dataframe``` dataframe. This function will allows us to use the API interface of the Central Bank of Reserve of Peru (BCRP) to automatically create a pandas dataframe with the necessary codes.

## 1.1 Libraries

We import the necessary libraries, including our own library in the modules file

In [1]:
# Warnings
import warnings
warnings.filterwarnings("ignore")

# Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from scipy import stats
from functools import reduce

# Statsmodels
import statsmodels.api as sm
import pmdarima as pmd
from pmdarima.arima import auto_arima
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VARResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL

# Machine Learning models
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, ElasticNet, ElasticNetCV, LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    median_absolute_error,
    r2_score,
    precision_score

)

from xgboost import XGBRegressor



In [2]:
# We import our own functions
import sys
sys.path.append('../../..')  # Move two levels up to the project root
from modules.functions import *

## 1.2 Extraction
We define our inputs and apply them the ```bcrp_dataframe``` function in order to obtain the pandas dataframe with the corresponding series

We define the following inputs:

    series     = the code of the series we are going to extract
    start_date = the starting date, when the BCRP starts using the interest rate as a policy measure
    end_date   = December 2019
    freq       = Monthly frequency

### df_1
We can now create the first dataframe with the ```bcrp_dataframe``` function. This dataframe contains all Consumer Price Index variables in monthly % change. This variables are then seasonally adjusted using the 

In [3]:
series     = ['PN01271PM', 'PN01280PM', 'PN01282PM', 'PN01278PM', 'PN09817PM','PN09816PM', 'PN01276PM', 'PN01313PM', 'PN01314PM',  
             'PN01315PM', 'PN09818PM','PN01286PM']
start_date = '2003-01'
end_date   = '2023-12'
freq       = 'Mensual'

In [4]:
df_1 = bcrp_dataframe( series , start_date , end_date , freq )
df_1.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Importado,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor
Fecha,,,,,,,,,,,,
2003-01-01,0.231420,0.179002,0.270381,-0.004192,0.593592,0.506054,-0.111733,0.398010,0.064330,-0.012549,0.702140,-0.161934
2003-02-01,0.468825,0.135374,0.708770,0.128185,0.992255,0.722192,0.228145,0.594648,0.406663,0.070296,0.727582,0.437078
2003-03-01,1.117780,0.400108,1.623551,0.157515,2.564665,0.357386,1.938313,-0.098522,2.136297,0.446856,1.391691,0.825606
2003-04-01,-0.050857,0.112646,-0.182128,0.074363,-0.260881,-0.265865,0.264059,-0.394477,0.305377,0.069202,0.275001,-0.212853
2003-05-01,-0.032025,-0.346022,0.181194,0.011232,-0.102900,-0.139973,0.055604,0.198020,-0.264172,0.177667,-1.113037,-0.141752


In [5]:
df_1 = get_trend(df_1)

df_1.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Importado,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor
Fecha,,,,,,,,,,,,
2003-01-01,0.092926,-0.090958,0.221921,0.023684,0.208767,-0.178262,0.421416,-0.272199,0.426209,0.110830,-0.264946,-0.193953
2003-02-01,0.112976,-0.053866,0.230003,0.029531,0.247979,-0.113158,0.387611,-0.195562,0.395215,0.106729,-0.165246,-0.130778
2003-03-01,0.133060,-0.016576,0.238007,0.035395,0.287271,-0.048449,0.354348,-0.120099,0.365339,0.102505,-0.066987,-0.068351
2003-04-01,0.153179,0.020809,0.246004,0.041241,0.326699,0.015669,0.321872,-0.046060,0.336814,0.098193,0.030109,-0.006735
2003-05-01,0.173188,0.058179,0.253822,0.047045,0.365928,0.078897,0.290215,0.026280,0.309608,0.093844,0.126013,0.053942


### df_2
We create the second dataframe with the ```bcrp_dataframe``` function. This dataframe contains rate variables. We use this variables in levels. It is not necessary to differentiate them.

In [6]:
series     = ['PN00493MM', 'PD04722MM']
start_date = '2003-09'
end_date   = '2023-12'
freq       = 'Mensual'

In [7]:
df_2 = bcrp_dataframe( series , start_date , end_date , freq )
df_2.head()

,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria
Fecha,,
2003-09-01,11.014920,2.75
2003-10-01,10.349944,2.75
2003-11-01,11.690608,2.50
2003-12-01,11.391178,2.50
2004-01-01,10.634030,2.50


### df_3
We create the third dataframe with the ```bcrp_dataframe``` function. This dataframe contains monetary variables. We differentiate those variables in order to get their monthly % change

In [8]:
series     = ['PN00495MM', 'PN06481IM', 'PN02125PM']
start_date = '2003-09'
end_date   = '2023-12'
freq       = 'Mensual'

In [9]:
df_3 = bcrp_dataframe( series , start_date , end_date , freq )
df_3 = df_3.pct_change()
df_3 = df_3.dropna()
df_3.head()

,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)
Fecha,,,
2003-10-01,0.010225,0.005542,0.056953
2003-11-01,0.011445,0.050288,-0.001674
2003-12-01,0.016607,-0.010532,-0.005608
2004-01-01,0.019490,0.036233,-0.005346
2004-02-01,0.017003,-0.005743,-0.010744


In [10]:
df = df_1.join(df_2).join(df_3)
df.dropna(inplace=True)
df.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Importado,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)
Fecha,,,,,,,,,,,,,,,,,
2003-10-01,0.266135,0.241045,0.283686,0.075596,0.545433,0.375007,0.140042,0.358268,0.186692,0.072083,0.575641,0.341826,10.349944,2.75,0.010225,0.005542,0.056953
2003-11-01,0.283860,0.277553,0.288313,0.081709,0.578708,0.433341,0.109075,0.422982,0.161919,0.067234,0.654246,0.396788,11.690608,2.50,0.011445,0.050288,-0.001674
2003-12-01,0.302840,0.315870,0.293891,0.088471,0.614307,0.494624,0.077290,0.489119,0.138211,0.061185,0.721827,0.449434,11.391178,2.50,0.016607,-0.010532,-0.005608
2004-01-01,0.315529,0.343327,0.296279,0.093899,0.636747,0.528569,0.064384,0.517058,0.137087,0.055042,0.754136,0.469576,10.634030,2.50,0.019490,0.036233,-0.005346
2004-02-01,0.324218,0.365682,0.295321,0.098356,0.650808,0.548055,0.060013,0.525733,0.145764,0.050551,0.781760,0.476891,10.718295,2.50,0.017003,-0.005743,-0.010744


In [11]:
df.tail()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Importado,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)
Fecha,,,,,,,,,,,,,,,,,
2023-08-01,0.182324,0.263821,0.147377,0.308731,-0.071459,0.175741,0.189515,0.263738,0.125240,0.215058,-0.170008,-0.104763,5.137951,7.75,-0.002584,-0.019404,-0.003747
2023-09-01,0.133933,0.237390,0.089467,0.281829,-0.163217,0.099223,0.165369,0.179682,0.102272,0.195769,-0.206148,-0.125837,5.269584,7.50,0.002271,-0.008615,-0.000164
2023-10-01,0.085365,0.210974,0.031289,0.254555,-0.254697,0.022078,0.141472,0.094521,0.079821,0.176261,-0.240150,-0.144366,5.277209,7.25,0.006684,-0.001206,0.003235
2023-11-01,0.036736,0.184472,-0.026935,0.226933,-0.345597,-0.055512,0.117886,0.008508,0.057913,0.156596,-0.272242,-0.160344,5.625714,7.00,-0.002776,0.008482,0.001633
2023-12-01,-0.011845,0.157822,-0.085018,0.198984,-0.435639,-0.133358,0.094639,-0.078087,0.036539,0.136819,-0.302719,-0.173843,5.857378,6.75,-0.013468,-0.010011,-0.003992


## 1.3 Data Inspection
We inspect the df. We first verify that all values are non-null. The, we apply the ```describe``` function to see the main variables.

In [12]:
df.isna().sum()

Índice de precios Lima Metropolitana (var% mensual) - IPC                                       0
Índice de precios Lima Metropolitana (var% mensual) - IPC Transables                            0
Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables                         0
Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente                            0
Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente                         0
Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía                   0
Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía               0
Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas                   0
Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas               0
Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas    0
Índice de precios Li

In [13]:
df.describe()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Importado,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)
count,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000
mean,0.264854,0.246279,0.276922,0.252162,0.286713,0.336332,0.198587,0.339784,0.209019,0.202923,0.273240,0.243250,9.770536,3.662551,0.010809,0.008591,0.001001
std,0.145368,0.150310,0.156400,0.111051,0.272234,0.234510,0.086266,0.240141,0.092416,0.077227,0.379287,0.280420,4.303846,1.712954,0.012282,0.028131,0.017831
min,-0.011845,-0.015181,-0.085018,0.075596,-0.435639,-0.133358,0.060013,-0.088401,0.036539,0.044367,-0.644228,-0.384483,5.000696,0.250000,-0.021338,-0.108487,-0.014610
25%,0.159071,0.142685,0.167191,0.162488,0.142472,0.168268,0.132115,0.163596,0.162233,0.139616,0.083366,0.084517,6.710217,2.750000,0.003853,-0.008683,-0.004463
50%,0.247743,0.211901,0.264602,0.254535,0.229393,0.296181,0.187431,0.298766,0.196458,0.203368,0.206620,0.165523,8.784346,3.750000,0.009649,0.004962,-0.002060
75%,0.319375,0.296747,0.347933,0.306935,0.384886,0.438704,0.236059,0.437529,0.237176,0.254538,0.368444,0.392430,10.882855,4.250000,0.017295,0.021158,-0.000187
max,0.679208,0.705393,0.711927,0.532530,1.065628,0.982719,0.458914,0.992529,0.514758,0.374857,1.374904,1.000380,26.380399,7.750000,0.074888,0.110080,0.130963


We have 195 observation ranging from ```2003-10-01``` to ```2019-12-01```. The mean of monthly % change of all CPI variables is around 0.2. The mean of the lacing rate and the interest rate is 10.7% and 3.67%, respectively. The three monetary variables have a small monthly % change, around 0.01 and 0.001 for Minimum Wage index. 

## 1.4 Data adjustment
We will rename the columns for easier identification of the variables. We will also create a new dataframe with the lags of the variables. 

In [14]:
# New column names
columns = {
    'Índice de precios Lima Metropolitana (var% mensual) - IPC': 'CPI',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Transables': 'CPI Tradable',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables': 'CPI Non-Tradable',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente': 'CPI Core',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente': 'CPI Non-Core',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía': 'CPI Food and Energy',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía': 'CPI Excluding Food and Energy',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas': 'CPI Food and Beverages',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas': 'CPI Excluding Food and Beverages',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas': 'CPI Core Excluding Food and Beverages',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Importado': 'CPI Imported',
    'Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor': 'Wholesale Price Index',
    'Tasas de interés del Banco Central de Reserva  - Tasa de Encaje': 'Reserve Requirement Rate',
    'Tasas de interés del Banco Central de Reserva  - Tasa de Referencia de la Política Monetaria': 'Monetary Policy Rate',
    'Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado': 'Circulating Currency Seasonally Adjusted (mill S/)',
    'Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$)': 'Net International Reserves (mill $)',
    'Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)': 'Real Minimum Wage (Index)'
}

# We rename the columns so they are easier to analyse
df.rename(columns=columns, inplace=True)

In [15]:
df_lags = df.copy()

for variable in df_lags.columns[1:]:
    df_lags[f'{variable}_lag_1'] = df_lags[variable].shift()
    df_lags[f'{variable}_lag_2'] = df_lags[variable].shift(2)

In [16]:
# We delete contemporary variables
df_lags.drop(columns = ['CPI Tradable', 'CPI Non-Tradable', 'CPI Core', 'CPI Non-Core', 'CPI Food and Energy', 'CPI Excluding Food and Energy',
       'CPI Food and Beverages', 'CPI Excluding Food and Beverages','CPI Core Excluding Food and Beverages', 'CPI Imported',
       'Wholesale Price Index', 'Reserve Requirement Rate','Monetary Policy Rate','Circulating Currency Seasonally Adjusted (mill S/)',
       'Net International Reserves (mill $)', 'Real Minimum Wage (Index)'], inplace = True)

df_lags = df_lags.dropna()

## 1.5 Save Results
We save it to the ```input``` folder, where we can use it to do the forecasting in the next notebook.

In [17]:
df.to_csv('../../../input/df_raw_h23.csv')

In [18]:
df_lags.to_csv('../../../input/df_lags_h23.csv')

In [19]:
df_lags.tail()

,CPI,CPI Tradable_lag_1,CPI Tradable_lag_2,CPI Non-Tradable_lag_1,CPI Non-Tradable_lag_2,CPI Core_lag_1,CPI Core_lag_2,CPI Non-Core_lag_1,CPI Non-Core_lag_2,CPI Food and Energy_lag_1,...,Reserve Requirement Rate_lag_1,Reserve Requirement Rate_lag_2,Monetary Policy Rate_lag_1,Monetary Policy Rate_lag_2,Circulating Currency Seasonally Adjusted (mill S/)_lag_1,Circulating Currency Seasonally Adjusted (mill S/)_lag_2,Net International Reserves (mill $)_lag_1,Net International Reserves (mill $)_lag_2,Real Minimum Wage (Index)_lag_1,Real Minimum Wage (Index)_lag_2
Fecha,,,,,,,,,,,,,,,,,,,,,
2023-08-01,0.182324,0.290249,0.316561,0.204813,0.261591,0.335223,0.361269,0.020204,0.111347,0.251427,...,5.617809,5.985237,7.75,7.75,-0.017678,-0.011038,0.004548,-0.043297,-0.003909,0.001566
2023-09-01,0.133933,0.263821,0.290249,0.147377,0.204813,0.308731,0.335223,-0.071459,0.020204,0.175741,...,5.137951,5.617809,7.75,7.75,-0.002584,-0.017678,-0.019404,0.004548,-0.003747,-0.003909
2023-10-01,0.085365,0.237390,0.263821,0.089467,0.147377,0.281829,0.308731,-0.163217,-0.071459,0.099223,...,5.269584,5.137951,7.50,7.75,0.002271,-0.002584,-0.008615,-0.019404,-0.000164,-0.003747
2023-11-01,0.036736,0.210974,0.237390,0.031289,0.089467,0.254555,0.281829,-0.254697,-0.163217,0.022078,...,5.277209,5.269584,7.25,7.50,0.006684,0.002271,-0.001206,-0.008615,0.003235,-0.000164
2023-12-01,-0.011845,0.184472,0.210974,-0.026935,0.031289,0.226933,0.254555,-0.345597,-0.254697,-0.055512,...,5.625714,5.277209,7.00,7.25,-0.002776,0.006684,0.008482,-0.001206,0.001633,0.003235


In [20]:
df.tail()

,CPI,CPI Tradable,CPI Non-Tradable,CPI Core,CPI Non-Core,CPI Food and Energy,CPI Excluding Food and Energy,CPI Food and Beverages,CPI Excluding Food and Beverages,CPI Core Excluding Food and Beverages,CPI Imported,Wholesale Price Index,Reserve Requirement Rate,Monetary Policy Rate,Circulating Currency Seasonally Adjusted (mill S/),Net International Reserves (mill $),Real Minimum Wage (Index)
Fecha,,,,,,,,,,,,,,,,,
2023-08-01,0.182324,0.263821,0.147377,0.308731,-0.071459,0.175741,0.189515,0.263738,0.125240,0.215058,-0.170008,-0.104763,5.137951,7.75,-0.002584,-0.019404,-0.003747
2023-09-01,0.133933,0.237390,0.089467,0.281829,-0.163217,0.099223,0.165369,0.179682,0.102272,0.195769,-0.206148,-0.125837,5.269584,7.50,0.002271,-0.008615,-0.000164
2023-10-01,0.085365,0.210974,0.031289,0.254555,-0.254697,0.022078,0.141472,0.094521,0.079821,0.176261,-0.240150,-0.144366,5.277209,7.25,0.006684,-0.001206,0.003235
2023-11-01,0.036736,0.184472,-0.026935,0.226933,-0.345597,-0.055512,0.117886,0.008508,0.057913,0.156596,-0.272242,-0.160344,5.625714,7.00,-0.002776,0.008482,0.001633
2023-12-01,-0.011845,0.157822,-0.085018,0.198984,-0.435639,-0.133358,0.094639,-0.078087,0.036539,0.136819,-0.302719,-0.173843,5.857378,6.75,-0.013468,-0.010011,-0.003992
